In [1]:
import pandas as pd 
import numpy as np 
import os
import torch
from torch import nn

# Deep Neural Networks Laboratory

## A first, shallow Neural Network

To check if the CUDA version of Pytorch was installed successfully, only if your machine supports CUDA, you can do:

In [2]:
torch.cuda.is_available()

True

The input of a neural network will be composed of two tensors: the sample and the label.

Imagine we have a classification task with 3 possible labels (0,1,2) and samples of 5 features (5-dimensional samples).

We will have tensors like:

In [3]:
x = torch.rand(1,5)
y = torch.randint(3, (1,)).long()

In [4]:
x

tensor([[0.8882, 0.8332, 0.9148, 0.9891, 0.5749]])

In [5]:
y

tensor([0])

Let us define a shallow, very simple Neural Network.

The Neural Network will have: 
1. As many input neurons as sample features (5 in this case)
2. An hidden layer 
3. As many ouput neurons as possible labels (3 in this case)

![images/shallow_nn.png](images/shallow_nn.png) 

This schema entails 4 different operations:
1. Matrix multiplication with the weights of the connections from the input layer to the FC Layer
2. Activation function in the FC Layer
3. Matrix multiplication with the weights of the connections from the FC Layer to the output layer
4. Activation function in the output layer

In [6]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.softmax(x)
        return x

## Step by step computation of a forward pass

So, we start with the tensor x.

In [7]:
x

tensor([[0.8882, 0.8332, 0.9148, 0.9891, 0.5749]])

x goes through the hidden layer. What this means is that we obtain the output:

$ Wx + b $ where:
1. W is the weight matrix of the connections
2. b is the bias vector

These values depend on the chosen initialization and might be random at first.

In [8]:
hidden_layer = nn.Linear(5,10)

print( hidden_layer.weight)

print( hidden_layer.bias )

Parameter containing:
tensor([[ 0.0046,  0.0711, -0.0436, -0.1085,  0.2851],
        [ 0.1946, -0.3188,  0.2857,  0.3318, -0.3322],
        [-0.2756, -0.1153,  0.0493,  0.4077,  0.3985],
        [-0.0564,  0.4144, -0.0344,  0.1715,  0.0687],
        [ 0.2608, -0.0114, -0.2764,  0.4133,  0.0580],
        [-0.4243, -0.1963,  0.1102, -0.2030, -0.2861],
        [ 0.0168,  0.2329, -0.0784,  0.2181,  0.2881],
        [-0.3238,  0.4046,  0.4036,  0.2073,  0.0044],
        [ 0.3768,  0.1875, -0.3480, -0.0174,  0.4319],
        [-0.0245, -0.0275,  0.2227, -0.3111, -0.2329]], requires_grad=True)
Parameter containing:
tensor([-0.1213,  0.0111, -0.3723,  0.0191,  0.1210,  0.4154,  0.1681, -0.0392,
        -0.1779, -0.3469], requires_grad=True)


In [9]:
w = hidden_layer.weight
b = hidden_layer.bias

output = torch.matmul(x, w.t()) + b
output

tensor([[-0.0412,  0.3169, -0.0357,  0.4919,  0.5324, -0.3895,  0.6868,  0.5871,
          0.2257, -0.6295]], grad_fn=<AddBackward0>)

If we apply the hidden layer, we obtain the same results:

In [10]:
hidden_layer(x)

tensor([[-0.0412,  0.3169, -0.0357,  0.4919,  0.5324, -0.3895,  0.6868,  0.5871,
          0.2257, -0.6295]], grad_fn=<AddmmBackward0>)

![images/dnn_2.jpeg](images/dnn_2.jpeg) 

In order to introduce non-linearity into our net, allowing it to learn more complex patterns, we pass the output of the FC layer through the ReLU activation function, which looks like this:


![images/relu.png](images/relu.png) 

In [11]:
output

tensor([[-0.0412,  0.3169, -0.0357,  0.4919,  0.5324, -0.3895,  0.6868,  0.5871,
          0.2257, -0.6295]], grad_fn=<AddBackward0>)

In [12]:
relu = nn.ReLU()

output = relu(output)
output

tensor([[0.0000, 0.3169, 0.0000, 0.4919, 0.5324, 0.0000, 0.6868, 0.5871, 0.2257,
         0.0000]], grad_fn=<ReluBackward0>)

In [13]:
relu(hidden_layer(x))

tensor([[0.0000, 0.3169, 0.0000, 0.4919, 0.5324, 0.0000, 0.6868, 0.5871, 0.2257,
         0.0000]], grad_fn=<ReluBackward0>)

Now, we go through another round of linear multiplication from the nodes inside the hidden layer to the nodes in the output layer.

In [14]:
output_layer = nn.Linear(10,3)

weights_2 = output_layer.weight 
bias_2 = output_layer.bias

output = torch.matmul(output, weights_2.t()) + bias_2
output

tensor([[ 0.1862,  0.4983, -0.1980]], grad_fn=<AddBackward0>)

In [15]:
output_layer(relu(hidden_layer(x)))

tensor([[ 0.1862,  0.4983, -0.1980]], grad_fn=<AddmmBackward0>)

We have obtained a tensor of the same size of the output layer, which is the same size of the number of possible labels in our classification task.


In order to convert these values to probabilities, we apply the SoftMax activation function.

![images/softmax.png](images/softmax.png) 

In [16]:
softmax = nn.Softmax(dim=1)

output = softmax(output)
output

tensor([[0.3282, 0.4484, 0.2235]], grad_fn=<SoftmaxBackward0>)

In [17]:
softmax(output_layer(relu(hidden_layer(x))))

tensor([[0.3282, 0.4484, 0.2235]], grad_fn=<SoftmaxBackward0>)

The outputs are the probabilities of the sample being labeled with each one of the labels (0,1,2).

We manually implemented a forward pass of the network we defined at the start:

In [18]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.softmax(x)
        return x

In [19]:
shallow_nn = ShallowNeuralNetwork(5,10,3)

The values might be slightly different due to the randomization of the starting weights:

In [20]:
shallow_nn(x)

tensor([[0.4034, 0.2297, 0.3669]], grad_fn=<SoftmaxBackward0>)